# Training SAEs with SAELens on Gemma 3

### Import libraries and detect hardware

In [2]:
import torch
import os
import gc
from transformers import AutoTokenizer, Gemma3ForCausalLM
from sae_lens import LanguageModelSAERunnerConfig, SAETrainingRunner

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print("Using device:", device)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

Using device: cuda


### Load Gemma model checkpoint from huggingface

In [3]:
ckpt = "google/gemma-3-4b-it"
tokenizer = AutoTokenizer.from_pretrained(ckpt)
model = Gemma3ForCausalLM.from_pretrained(
    ckpt,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.32s/it]


### Train the SAE

In [ ]:
total_training_steps = 30_000
batch_size = 4096
total_training_tokens = total_training_steps * batch_size

lr_warm_up_steps = 0
lr_decay_steps = total_training_steps // 5
l1_warm_up_steps = total_training_steps // 20

cfg = LanguageModelSAERunnerConfig(
    # SAE architecture and model
    architecture="topk",  # topk used since matryoshka batch topk converts to jumprelu
    activation_fn="topk",
    activation_fn_kwargs={"k": 40},  # important: this is where 'k' goes
    model_name="google/gemma-3-1b-pt", # lol it wasn't working for so long because it didn't have google in front of it 
    model_class_name="AutoModelForCausalLM",
    hook_name="model.layers.0",
    hook_layer=0,
    d_in=1152,
    d_sae=32768,

    # Dataset
    dataset_path="apollo-research/monology-pile-uncopyrighted-tokenizer-EleutherAI-gpt-neox-20b",
    dataset_trust_remote_code=True,
    streaming=True,
    context_size=2048,
    prepend_bos=True,

    # Training
    lr=3e-4,
    adam_beta1=0.9,
    adam_beta2=0.999,
    lr_scheduler_name="constant",
    lr_warm_up_steps=lr_warm_up_steps,
    lr_decay_steps=lr_decay_steps,
    l1_coefficient=0.001,
    l1_warm_up_steps=l1_warm_up_steps,
    lp_norm=1.0,
    train_batch_size_tokens=batch_size,
    training_tokens=total_training_tokens,
    n_batches_in_buffer=4,
    store_batch_size_prompts=2,

    # Init + Heuristics
    apply_b_dec_to_input=True,
    decoder_orthogonal_init=False,
    decoder_heuristic_init=True,
    init_encoder_as_decoder_transpose=True,
    normalize_sae_decoder=False,
    normalize_activations="none",
    exclude_special_tokens=True,

    # Logging
    log_to_wandb=True,
    wandb_project="sae_gemma3_experiment",
    wandb_log_frequency=30,
    eval_every_n_wandb_logs=20,

    # Misc
    device="cuda",
    act_store_device="with_model",
    seed=42,
    n_checkpoints=0,
    checkpoint_path="checkpoints",
    verbose=True,

    # Stability
    use_ghost_grads=False,
    feature_sampling_window=1000,
    dead_feature_window=1000,
    dead_feature_threshold=1e-4
)

sparse_autoencoder = SAETrainingRunner(cfg).run()


Objective value: 2408587.0000:   8%|▊         | 8/100 [00:00<00:00, 725.52it/s]
/home/henrytsai/anaconda3/envs/henry/lib/python3.10/site-packages/sae_lens/training/training_sae.py:636: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  out = torch.tensor(origin, dtype=self.dtype, device=self.device)
wandb: Currently logged in as: htsai (htsai2025) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


200| auxiliary_reconstruction_loss: 0.00000 | mse_loss: 11170.60254:   1%|          | 819200/122880000 [02:36<5:49:45, 5816.30it/s]

### [Deletes the model] Clean up the GPU memory

In [ ]:
del sparse_autoencoder  # or whatever large object you want to remove
gc.collect()
torch.cuda.empty_cache()

: 